In [1]:
import json
import random

from datetime import datetime
import time

In [2]:
with open('./json_agenti.json') as json_file:
    agenti = json.load(json_file)
with open('./json_info_sezioni.json') as json_file:
    info_sezioni = json.load(json_file)

In [4]:
#Controllo consistenza dati
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

for s in info_sezioni:
    agenti_residenti=list(filter(lambda a: a['sez_residenza'] == s['sezione'], agenti))
    if s['abitanti']<len(agenti_residenti):
        print("Numero residenti sezione: "+str(s['abitanti'])+"Abitanti necessari: "+str(len(agenti_residenti)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-23 15:14:05
Fine = 2021-03-23 15:16:47


In [5]:
#Creazione nucli famigliari
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

id_fam=0
agenti_new=[]

for a in agenti:
    if not 'id_fam' in a and a['tipo']=='attiva':
        tipi=list(range(0,6))
        random.shuffle(tipi)
    
        for t in tipi:
            if t==0: #Single           
                a['id_fam']=id_fam
                id_fam=id_fam+1
                agenti_new.append(a)
                break
                
            if t==1: #Coppia
                a['id_fam']=id_fam
                agenti_new.append(a)
                agenti_residenti=list(filter(lambda an: an['sez_residenza'] == a['sez_residenza'] and an['tipo']=='attiva' and not 'id_fam' in an,agenti))
                try:
                    a2=agenti_residenti[0]
                    a2['id_fam']=id_fam
                    agenti_new.append(a2)
                    id_fam=id_fam+1
                    break
                except:
                    id_fam=id_fam+1
                    break
            if t==2: #Coppia con figlio piccolo
                a['id_fam']=id_fam
                agenti_new.append(a)
                
                #Partner
                agenti_residenti=list(filter(lambda an: an['sez_residenza'] == a['sez_residenza'] and an['tipo']=='attiva' and not 'id_fam' in an,agenti))
                try:
                    a2=agenti_residenti[0]
                    a2['id_fam']=id_fam
                    agenti_new.append(a2)
                except:
                    id_fam=id_fam+1
                    break
                #Figlio 
                agenti_residenti=list(filter(lambda an: an['sez_residenza'] == a['sez_residenza'] and an['tipo']=='giovane' and not 'id_fam' in an,agenti))
                try:
                    g1=agenti_residenti[0]
                    g1['id_fam']=id_fam
                    agenti_new.append(g1)
                    id_fam=id_fam+1
                    break
                except:
                    id_fam=id_fam+1
                    break
            if t==3: #Single con figlio
                a['id_fam']=id_fam
                agenti_new.append(a)
                
                #Figlio 
                agenti_residenti=list(filter(lambda an: an['sez_residenza'] == a['sez_residenza'] and an['tipo']=='giovane' and not 'id_fam' in an,agenti))
                try:
                    g1=agenti_residenti[0]
                    g1['id_fam']=id_fam
                    agenti_new.append(g1)
                    id_fam=id_fam+1
                    break
                except:
                    id_fam=id_fam+1
                    break
            if t==4: #Coppia con figlio attivo
                a['id_fam']=id_fam
                agenti_new.append(a)
                
                #Partner
                agenti_residenti=list(filter(lambda an: an['sez_residenza'] == a['sez_residenza'] and an['tipo']=='attiva' and not 'id_fam' in an,agenti))
                try:
                    a2=agenti_residenti[0]
                    a2['id_fam']=id_fam
                    agenti_new.append(a2)
                except:
                    id_fam=id_fam+1
                    break
                    
                #Figlio 
                agenti_residenti=list(filter(lambda an: an['sez_residenza'] == a['sez_residenza'] and an['categoria']=='studente' and not 'id_fam' in an,agenti))
                try:
                    s1=agenti_residenti[0]
                    s1['id_fam']=id_fam
                    agenti_new.append(s1)
                    id_fam=id_fam+1
                    break
                except:
                    id_fam=id_fam+1
                    break
            if t==5: #Studenti (max 3)
                a['id_fam']=id_fam
                agenti_new.append(a)
                
                #Studente 2
                agenti_residenti=list(filter(lambda an: an['sez_residenza'] == a['sez_residenza'] and an['categoria']=='studente' and not 'id_fam' in an,agenti))
                try:
                    s1=agenti_residenti[0]
                    s1['id_fam']=id_fam
                    agenti_new.append(s1)
                except:
                    id_fam=id_fam+1
                    break
                #Studente 3 
                agenti_residenti=list(filter(lambda an: an['sez_residenza'] == a['sez_residenza'] and an['categoria']=='studente' and not 'id_fam' in an,agenti))
                try:
                    s2=agenti_residenti[0]
                    s2['id_fam']=id_fam
                    agenti_new.append(s2)
                    id_fam=id_fam+1
                    break
                except:
                    id_fam=id_fam+1
                    break

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))


Inizio = 2021-03-23 15:17:47
Fine = 2021-03-23 16:21:37


In [6]:
#Salvo il file
with open('./json_agenti_fam.json', 'w') as fp:
        json.dump(agenti_new, fp)

In [7]:
#Controllo elaborazione - Counter numero di nuclei per grandezza
id_fam_c=0
count=0
tipo=[]
for a in agenti_new:
    id_fam=a['id_fam']
    if id_fam_c==id_fam:
        count=count+1
    else:
        tipo.append(count)
        id_fam_c=id_fam
        count=1
        
from collections import Counter

Counter(tipo)
    

Counter({1: 34698, 2: 41801, 3: 28759})